In [23]:
import xml.etree.ElementTree as ET

In [58]:
import csv
import xml.etree.ElementTree as ET

# List of XML file paths
xml_file_paths = ['../resources/grobid/Level1_combined.grobid.tei.xml', '../resources/grobid/Level2_combined.grobid.tei.xml','../resources/grobid/Level3_combined.grobid.tei.xml']

# Create a CSV file
csv_filename = '../resources/grobid/outcomes.csv'

with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write header
    csv_writer.writerow(['Topic', 'Learning Outcomes Section'])
    
    # Iterate through each XML file
    for xml_file_path in xml_file_paths:
        print(xml_file_path)
        # Read the XML data from the file
        with open(xml_file_path, 'r') as file:
            xml_data = file.read()
        # Parse the XML data
        root = ET.fromstring(xml_data)

        for div_element in root.findall('.//{http://www.tei-c.org/ns/1.0}div'):
                head_element = div_element.find('.//{http://www.tei-c.org/ns/1.0}head')

                p_elements = div_element.findall('.//{http://www.tei-c.org/ns/1.0}p')

                combined_p_text = ' '.join(p_element.text for p_element in p_elements if p_element.text)
                if combined_p_text != '':
                    csv_writer.writerow([head_element.text if head_element is not None else '', combined_p_text])

print(f"Combined CSV file '{csv_filename}' created successfully.")



../resources/grobid/Level1_combined.grobid.tei.xml
../resources/grobid/Level2_combined.grobid.tei.xml
../resources/grobid/Level3_combined.grobid.tei.xml
Combined CSV file '../resources/grobid/outcomes.csv' created successfully.


In [25]:
import os
from dotenv import load_dotenv
load_dotenv()

print(os.getenv('snowflake_account'))

jr53964.us-east-2.aws


In [59]:
import pandas as pd
import snowflake.connector

snowflake_account = os.getenv('snowflake_account')
snowflake_user = os.getenv('snowflake_user')
snowflake_password = os.getenv('snowflake_password')
snowflake_warehouse = os.getenv('snowflake_warehouse')
snowflake_schema = os.getenv('snowflake_schema')


conn = snowflake.connector.connect(
    user=snowflake_user,
    password=snowflake_password,
    account=snowflake_account,
    warehouse=snowflake_warehouse,
    schema=snowflake_schema
)



cursor = conn.cursor()

target_database = 'CFAInstitute'
target_table = 'Learning_Outcomes'

create_database_query = f"CREATE DATABASE IF NOT EXISTS {target_database}"
cursor.execute(create_database_query)

use_database_query = f"USE DATABASE {target_database}"
cursor.execute(use_database_query)
cursor.execute("USE WAREHOUSE TEST")

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {target_table} (
    "Learning Outcomes Section" VARCHAR,
    Topic VARCHAR
)
"""

cursor.execute(create_table_query)


In [14]:
# cursor.execute(f"TRUNCATE TABLE {target_table}")

# cursor.execute(f"PUT file://combined_output.csv @%{target_table}")

# cursor.execute(f"COPY INTO {target_table} ON_ERROR=CONTINUE FILE_FORMAT = (FIELD_OPTIONALLY_ENCLOSED_BY = '\"' SKIP_HEADER=1 PARSE_HEADER = FALSE)")


# Process and upload to snowflake

In [61]:
def remove_extra_whitespaces(value):
    if isinstance(value, str):
        return ' '.join(value.split())
    else:
        return value

def preprocess_text(value):
    if isinstance(value, str):
        return f'"{value}"'
    else:
        return value




# Upload to s3 first now

In [62]:
import boto3

def upload_folder_to_s3(local_folder_path, s3_bucket_name, s3_folder_path, aws_access_key_id, aws_secret_access_key):
    # Create an S3 client
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

    # Iterate through all the files in the local folder
    for root, dirs, files in os.walk(local_folder_path):
        for file in files:
            local_file_path = os.path.join(root, file)
            s3_key = os.path.relpath(local_file_path, local_folder_path)
            s3_object_key = os.path.join(s3_folder_path, s3_key).replace("\\", "/")  # Replace backslashes with forward slashes for cross-platform compatibility
            
            # Upload each file to S3
            print(f"Uploading {local_file_path} to s3://{s3_bucket_name}/{s3_object_key}")
            s3.upload_file(local_file_path, s3_bucket_name, s3_object_key)

local_folder_path = '../resources/grobid/out'
aws_access_key_id = os.getenv('aws_access_key_id')
aws_secret_access_key = os.getenv('aws_secret_access_key')
bucket_name = 'cfainstitute-learning-outcomes-raw'
s3_folder_path = 'grobid'
upload_folder_to_s3(local_folder_path, bucket_name, s3_folder_path, aws_access_key_id, aws_secret_access_key,)


Uploading ../resources/grobid/out/Grobid_RR_2024_Level3_combined.txt to s3://cfainstitute-learning-outcomes-raw/grobid/Grobid_RR_2024_Level3_combined.txt
Uploading ../resources/grobid/out/Grobid_RR_2024_Level2_combined.txt to s3://cfainstitute-learning-outcomes-raw/grobid/Grobid_RR_2024_Level2_combined.txt
Uploading ../resources/grobid/out/Grobid_RR_2024_Level1_combined.txt to s3://cfainstitute-learning-outcomes-raw/grobid/Grobid_RR_2024_Level1_combined.txt


Upload outcomes.csv to s3

In [63]:
import pandas as pd

df = pd.read_csv('../resources/grobid/outcomes.csv', dtype=str)
df = df.map(remove_extra_whitespaces)
df = df.map(preprocess_text)

df.to_csv('../resources/grobid/outcomes_processed.csv', index=False)

s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

csv_data = df.to_csv(index=False)

s3_key = 'outcomes_processed.csv'

s3.put_object(Body=csv_data, Bucket=bucket_name, Key=s3_key)

print(f"CSV file has been uploaded to S3 at s3://{bucket_name}/{s3_key}")


CSV file has been uploaded to S3 at s3://cfainstitute-learning-outcomes-raw/outcomes_processed.csv


# Create file format first

In [64]:
cursor.execute("""CREATE OR REPLACE FILE FORMAT mycsvformat
   TYPE = 'CSV'
   FIELD_DELIMITER = '|'
   SKIP_HEADER = 1;""")

 Goto iam of aws and create a role
 use above role arn to create storage integration
 Create STORAGE INTEGRATION that can connect to aws account

In [32]:
# cursor.execute("""CREATE OR REPLACE STORAGE INTEGRATION s3_int2
#   TYPE = EXTERNAL_STAGE
#   STORAGE_PROVIDER = 'S3'
#   STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::640055273174:role/s3-read'
#   ENABLED = TRUE
#   STORAGE_ALLOWED_LOCATIONS = ('*')
# """)

 go to trusted relationships of iam role created in above step from aws console and change it to below template

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "s3.amazonaws.com",
                "AWS": "<user arn>"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "sts:ExternalId": "<external id>"
                }
            }
        }
    ]
}

run "DESC Integration s3_int2"  inside snowflake to get STORAGE_AWS_IAM_USER_ARN and STORAGE_AWS_EXTERNAL_ID of storage integration crreated

In [65]:
cursor.execute("""CREATE OR REPLACE STAGE external_stage
  FILE_FORMAT = mycsvformat
  URL = 's3://cfainstitute-learning-outcomes-raw/outcomes.csv'
  STORAGE_INTEGRATION = s3_int2;
""")

In [ ]:
# cursor.execute(f"""
# COPY INTO {target_table}
#   FROM @external_stage
#   ON_ERROR = 'CONTINUE';
# """)

In [ ]:
# file_format = (type = csv skip_header = 1  PARSE_HEADER = FALSE field_delimiter = ',');


In [ ]:
# Upload to snowflake from s3

In [70]:
cursor.execute("""
copy into Learning_Outcomes
from 's3://cfainstitute-learning-outcomes-raw/outcomes_processed.csv'
storage_integration = s3_int2
FORCE = TRUE
ON_ERROR = CONTINUE
  file_format = (type = csv FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER=1 PARSE_HEADER = FALSE);
""")